In [12]:
import numpy as np
import seaborn as sns
import torchsummary
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torch.nn import Linear, Sequential, Dropout, ReLU
from torch import load, inference_mode, round, sigmoid, randn
from sklearn.metrics import accuracy_score, multilabel_confusion_matrix, classification_report
from torchvision.models.regnet import regnet_y_3_2gf, RegNet_Y_3_2GF_Weights
from torchvision.models.swin_transformer import swin_v2_t, Swin_V2_T_Weights
from torchvision.models.efficientnet import efficientnet_v2_s, EfficientNet_V2_S_Weights

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
regnet_model = regnet_y_3_2gf()
regnet_model.fc = Linear(1512, 28)
regnet_model.load_state_dict(load("../Models/FinetunedRegNet.pth"))

<All keys matched successfully>

In [4]:
efficientnet_model = efficientnet_v2_s()
efficientnet_model.classifier = Sequential(
    Dropout(p=0.2),
    ReLU(),
    Linear(in_features=1280, out_features=28)
)
efficientnet_model.load_state_dict(load("../Models/FinetunedEfficientNet.pth"))

<All keys matched successfully>

In [5]:
swinv2_model = swin_v2_t()
swinv2_model.head = Linear(768, 28)
swinv2_model.load_state_dict(load("../Models/FinetunedSwinV2.pth"))

<All keys matched successfully>

In [15]:
torchsummary.summary(regnet_model.to('cuda'),(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             864
       BatchNorm2d-2         [-1, 32, 112, 112]              64
              ReLU-3         [-1, 32, 112, 112]               0
            Conv2d-4           [-1, 72, 56, 56]           2,304
       BatchNorm2d-5           [-1, 72, 56, 56]             144
            Conv2d-6         [-1, 72, 112, 112]           2,304
       BatchNorm2d-7         [-1, 72, 112, 112]             144
              ReLU-8         [-1, 72, 112, 112]               0
            Conv2d-9           [-1, 72, 56, 56]          15,552
      BatchNorm2d-10           [-1, 72, 56, 56]             144
             ReLU-11           [-1, 72, 56, 56]               0
AdaptiveAvgPool2d-12             [-1, 72, 1, 1]               0
           Conv2d-13              [-1, 8, 1, 1]             584
             ReLU-14              [-1, 

In [16]:
torchsummary.summary(efficientnet_model.to('cuda'), (3,384,384))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 24, 192, 192]             648
       BatchNorm2d-2         [-1, 24, 192, 192]              48
              SiLU-3         [-1, 24, 192, 192]               0
            Conv2d-4         [-1, 24, 192, 192]           5,184
       BatchNorm2d-5         [-1, 24, 192, 192]              48
              SiLU-6         [-1, 24, 192, 192]               0
   StochasticDepth-7         [-1, 24, 192, 192]               0
       FusedMBConv-8         [-1, 24, 192, 192]               0
            Conv2d-9         [-1, 24, 192, 192]           5,184
      BatchNorm2d-10         [-1, 24, 192, 192]              48
             SiLU-11         [-1, 24, 192, 192]               0
  StochasticDepth-12         [-1, 24, 192, 192]               0
      FusedMBConv-13         [-1, 24, 192, 192]               0
           Conv2d-14           [-1, 96,

In [17]:
torchsummary.summary(swinv2_model.to("cuda"), (3,260,260))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 65, 65]           4,704
           Permute-2           [-1, 65, 65, 96]               0
         LayerNorm-3           [-1, 65, 65, 96]             192
            Linear-4          [-1, 15, 15, 512]           1,536
              ReLU-5          [-1, 15, 15, 512]               0
            Linear-6            [-1, 15, 15, 3]           1,536
ShiftedWindowAttentionV2-7           [-1, 65, 65, 96]               0
         LayerNorm-8           [-1, 65, 65, 96]             192
   StochasticDepth-9           [-1, 65, 65, 96]               0
           Linear-10          [-1, 65, 65, 384]          37,248
             GELU-11          [-1, 65, 65, 384]               0
          Dropout-12          [-1, 65, 65, 384]               0
           Linear-13           [-1, 65, 65, 96]          36,960
          Dropout-14           [-

In [18]:
regnet_model

RegNet(
  (stem): SimpleStemIN(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (trunk_output): Sequential(
    (block1): AnyStage(
      (block1-0): ResBottleneckBlock(
        (proj): Conv2dNormActivation(
          (0): Conv2d(32, 72, kernel_size=(1, 1), stride=(2, 2), bias=False)
          (1): BatchNorm2d(72, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (f): BottleneckTransform(
          (a): Conv2dNormActivation(
            (0): Conv2d(32, 72, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): BatchNorm2d(72, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU(inplace=True)
          )
          (b): Conv2dNormActivation(
            (0): Conv2d(72, 72, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=3, bias=False)
            

In [19]:
efficientnet_model

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  

In [20]:
swinv2_model

SwinTransformer(
  (features): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
      (1): Permute()
      (2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
    )
    (1): Sequential(
      (0): SwinTransformerBlockV2(
        (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
        (attn): ShiftedWindowAttentionV2(
          (qkv): Linear(in_features=96, out_features=288, bias=True)
          (proj): Linear(in_features=96, out_features=96, bias=True)
          (cpb_mlp): Sequential(
            (0): Linear(in_features=2, out_features=512, bias=True)
            (1): ReLU(inplace=True)
            (2): Linear(in_features=512, out_features=3, bias=False)
          )
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
        (norm2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (0): Linear(in_features=96, out_features=384, bias=True)
          (1): GELU(approximate='